In [6]:
import pandas as pd

In [58]:
df = pd.read_csv('data.csv')
handles = df[df['platform'] == 'Twitter']['handle'].to_list()

582

In [ ]:
# from tweeterpy import TweeterPy

# twitter = TweeterPy()

# (twitter.get_user_data('elonmusk'))

✓ Loaded 155 cached profiles from twitter_user_cache.csv
2025-12-09 01:02:44,098 [WARNING] :: Skipping API Updates. Couldn't get the latest API data.
2025-12-09 01:02:44,101 [INFO] :: API Updated Successfully.
2025-12-09 01:02:44,098 [WARNING] :: Skipping API Updates. Couldn't get the latest API data.
2025-12-09 01:02:44,101 [INFO] :: API Updated Successfully.


✓ Loaded 155 cached profiles from twitter_user_cache.csv
2025-12-09 01:02:44,098 [WARNING] :: Skipping API Updates. Couldn't get the latest API data.
2025-12-09 01:02:44,101 [INFO] :: API Updated Successfully.
2025-12-09 01:02:44,098 [WARNING] :: Skipping API Updates. Couldn't get the latest API data.
2025-12-09 01:02:44,101 [INFO] :: API Updated Successfully.


KeyError: 'followers'

In [62]:
from tweeterpy import TweeterPy

def load_twitter_cache(cache_file='twitter_cache.csv'):
    """Load cached Twitter data from CSV file."""
    if os.path.exists(cache_file):
        try:
            cache_df = pd.read_csv(cache_file)
            print(f"✓ Loaded {len(cache_df)} cached profiles from {cache_file}")
            return cache_df
        except Exception as e:
            print(f"Warning: Could not load cache file: {e}")
            return pd.DataFrame()
    else:
        print(f"No cache file found. Will create {cache_file}")
        return pd.DataFrame()

def save_twitter_cache(data, cache_file='twitter_cache.csv'):
    """Save Twitter data to CSV cache file."""
    try:
        df = pd.DataFrame(data)
        df.to_csv(cache_file, index=False)
        print(f"✓ Saved {len(df)} profiles to {cache_file}")
    except Exception as e:
        print(f"Warning: Could not save cache: {e}")

def get_twitter_user_data(handle, twitter):
    """Fetch Twitter user data for a single handle."""
    try:
        user_data = twitter.get_user_data(handle)
        
        if user_data:
            result = {
                'handle': handle,
                'user_data': str(user_data),  # Store the entire user data object as string
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            return result
        else:
            print(f"No data returned for {handle}")
            return None
            
    except Exception as e:
        print(f"Error fetching data for {handle}: {e}")
        return None

def get_all_twitter_data(handles, delay=3, cache_file='twitter_cache.csv', use_cache=True):
    """
    Fetch Twitter data for multiple handles with caching.
    
    Args:
        handles: List of Twitter handles
        delay: Seconds to wait between requests (default: 3)
        cache_file: Path to cache file
        use_cache: Whether to use cached data
    """
    # Load cache
    cache_df = load_twitter_cache(cache_file) if use_cache else pd.DataFrame()
    
    # Initialize Twitter client
    twitter = TweeterPy()
    
    results = []
    cached_handles = set(cache_df['handle'].tolist()) if not cache_df.empty and 'handle' in cache_df.columns else set()
    
    # Add cached data to results
    if use_cache and not cache_df.empty:
        for handle in handles:
            if handle in cached_handles:
                cached_row = cache_df[cache_df['handle'] == handle].iloc[0].to_dict()
                results.append(cached_row)
                print(f"[CACHED] @{handle}")
    
    # Get handles that need to be fetched
    handles_to_fetch = [h for h in handles if h not in cached_handles]
    
    if handles_to_fetch:
        print(f"\n📊 Need to fetch {len(handles_to_fetch)} new profiles")
    else:
        print(f"\n✅ All {len(handles)} profiles loaded from cache!")
    
    # Fetch new data
    for i, handle in enumerate(handles_to_fetch):
        print(f"\nFetching data for ({i+1}/{len(handles_to_fetch)}): @{handle}")
        
        data = get_twitter_user_data(handle, twitter)
        
        if data:
            results.append(data)
            print(f"✓ @{data['handle']}: Data retrieved")
            
            # Save cache after each successful fetch
            save_twitter_cache(results, cache_file)
        else:
            print(f"✗ Failed to fetch data for @{handle}")
        
        # Add delay between requests
        if i < len(handles_to_fetch) - 1:
            print(f"Waiting {delay} seconds before next request...")
            time.sleep(delay)
    
    return results

# Fetch data for all handles
twitter_data = get_all_twitter_data(handles, delay=5, cache_file='twitter_user_cache.csv', use_cache=True)

✓ Loaded 443 cached profiles from twitter_user_cache.csv
2025-12-09 01:33:25,796 [WARNING] :: Skipping API Updates. Couldn't get the latest API data.
2025-12-09 01:33:25,800 [INFO] :: API Updated Successfully.
[CACHED] @GovDunleavy
[CACHED] @GovernorKayIvey
[CACHED] @SarahHuckabee
[CACHED] @GovernorHobbs
[CACHED] @CAgovernor
[CACHED] @GovofCO
[CACHED] @GovNedLamont
[CACHED] @MattMeyerDE
[CACHED] @GovRonDeSantis
[CACHED] @GovKemp
[CACHED] @GovJoshGreenMD
[CACHED] @IAGovernor
[CACHED] @GovernorLittle
[CACHED] @GovPritzker
[CACHED] @GovBraun
[CACHED] @GovLauraKelly
[CACHED] @GovAndyBeshear
[CACHED] @LAGovJeffLandry
[CACHED] @GovWesMoore
[CACHED] @GovJanetMills
[CACHED] @GovWhitmer
[CACHED] @GovTimWalz
[CACHED] @GovMikeKehoe
[CACHED] @tatereeves
[CACHED] @NC_Governor
[CACHED] @GovArmstrong
[CACHED] @TeamPillen
[CACHED] @KellyAyotte
[CACHED] @GovMLG
[CACHED] @JosephMLombardo
[CACHED] @GovKathyHochul
[CACHED] @GovMikeDeWine
[CACHED] @GovStitt
[CACHED] @GovTinaKotek
[CACHED] @GovernorShapiro


In [70]:
twitter_data_df = pd.DataFrame(twitter_data)
